In [27]:
import subprocess
import os
import glob
import numpy as np
import pandas as pd
from time import process_time as ptime

In [2]:
# ffmpeg binary
ffmpeg = "/usr/bin/ffmpeg"

# text directory
text_dir = "../data/raw/text"

# labels file
labels_file = "../data/interim/labels/interim.csv"

# audio directories
audio_dir = "../data/raw/audio"
audio_out_dir = "../data/interim/audio"

## Load aggregated labels

In [3]:
df_labels = pd.read_csv(labels_file, sep=',', header=0)
print(df_labels.shape)
df_labels.head()

(21865, 8)


,id,anger,disgust,fear,happiness,sadness,surprise,sentiment
0,--qXJuDtHPw_5,0,0,0,1,0,0,1
1,-3g5yACwYnA_10,0,0,0,0,0,0,1
2,-3g5yACwYnA_13,0,0,0,0,0,0,1
3,-3g5yACwYnA_2,0,0,0,1,0,0,0
4,-3g5yACwYnA_3,0,0,0,0,0,0,0


## Split clip from id for audio processing

In [6]:
df_labels['clip'] = \
    df_labels['id'].map(lambda x : x.rsplit('_', maxsplit=1)[1])
df_labels['id'] = \
    df_labels['id'].map(lambda x : x.rsplit('_', maxsplit=1)[0])
df_labels.head()

,id,anger,disgust,fear,happiness,sadness,surprise,sentiment,clip
0,--qXJuDtHPw,0,0,0,1,0,0,1,5
1,-3g5yACwYnA,0,0,0,0,0,0,1,10
2,-3g5yACwYnA,0,0,0,0,0,0,1,13
3,-3g5yACwYnA,0,0,0,1,0,0,0,2
4,-3g5yACwYnA,0,0,0,0,0,0,0,3


## Reorder columns

In [7]:
labels_cols = df_labels.columns.to_list()
print(labels_cols)

['id', 'anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'sentiment', 'clip']


In [8]:
cols = ['id', 'clip', 'anger', 'disgust', 'fear',
        'happiness', 'sadness', 'surprise', 'sentiment']
df_labels = df_labels[cols]
df_labels.head(3)

,id,clip,anger,disgust,fear,happiness,sadness,surprise,sentiment
0,--qXJuDtHPw,5,0,0,0,1,0,0,1
1,-3g5yACwYnA,10,0,0,0,0,0,0,1
2,-3g5yACwYnA,13,0,0,0,0,0,0,1


## Create dict with labeled clips

In [45]:
labeled_id_clips = {}

uniq_ids = sorted(list(df_labels['id'].unique()))
for i in uniq_ids:
    labeled_id_clips[i] = \
        sorted(
            df_labels[df_labels['id'] == i]['clip'].astype(int).to_list())

In [46]:
print(labeled_id_clips['-3g5yACwYnA'])

[2, 3, 4, 9, 10, 13]


In [30]:
def get_audio_info_from_text(text_dir):
    '''
        get audio information from text files to extract clips
        - clip number
        - clip start time
        - clip end time
        return DataFrame
    '''
    text_files = glob.glob(f"{text_dir}/*.txt")
    df_list = []
    for file in text_files:
        with open(file, 'r') as text_file:
            text_lines = text_file.readlines()
            #df_text = pd.DataFrame()
            text_dict_list = []
            for text_line in text_lines:
                line_dict = {}
                split_line = text_line.split('___')
                line_dict['id'] = split_line[0]
                line_dict['clip'] = int(split_line[1])
                line_dict['start_time'] = abs(float(split_line[2]))
                line_dict['end_time'] = float(split_line[3])
                
                # leave text out for now
                # line_dict['text'] = split_line[4][:-1]
                
                text_dict_list.append(line_dict)
        df_list.append(pd.DataFrame(text_dict_list))
    df = pd.concat(df_list, axis=0, ignore_index=True)
    return df

In [31]:
df_audio_info = get_audio_info_from_text(text_dir)
df_audio_info.head()

,id,clip,start_time,end_time
0,xYtNgq9eTlg,0,0.000,17.943
1,xYtNgq9eTlg,1,17.342,25.376
2,xYtNgq9eTlg,2,24.755,30.255
3,xYtNgq9eTlg,3,29.554,36.331
4,xYtNgq9eTlg,4,35.700,39.494


In [53]:
def get_labeled_clip_info(df_audio_info, labeled_id_clips):
    labeled_clips_info = []
    for i, clips in labeled_id_clips.items():
        mask_id = df_audio_info['id'] == i
        for clip in clips:
            mask_clip = df_audio_info['clip'] == clip
            clip_info = {}
            clip_info['id'] = i
            clip_info['clip'] = clip
            try:
                clip_info['start_time'] = \
                    df_audio_info[(mask_id) & (mask_clip)]['start_time'].values[0]
            except:
                print(df_audio_info[(mask_id) & (mask_clip)])
            try:
                clip_info['end_time'] = \
                    df_audio_info[(mask_id) & (mask_clip)]['end_time'].values[0]
            except:
                print(df_audio_info[(mask_id) & (mask_clip)])
            labeled_clips_info.append(clip_info)
    df_clip_info = pd.DataFrame(labeled_clips_info)
    return df_clip_info

# Check error ??? index 0 not in axis

In [54]:
df_clip_info = get_labeled_clip_info(df_audio_info, labeled_id_clips)
df_clip_info.head(10)

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

...

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, end_time]
Index: []
Empty DataFrame
Columns: [id, clip, start_time, 

,id,clip,start_time,end_time
0,--qXJuDtHPw,5,23.199,30.325
1,-3g5yACwYnA,2,4.840,14.052
2,-3g5yACwYnA,3,13.211,27.521
3,-3g5yACwYnA,4,26.541,41.300
4,-3g5yACwYnA,9,74.083,82.776
5,-3g5yACwYnA,10,82.753,100.555
6,-3g5yACwYnA,13,119.919,125.299
7,-3nNcZdcdvU,0,0.000,9.213
8,-3nNcZdcdvU,2,12.014,20.098
9,-571d8cVauQ,0,0.000,4.992


In [55]:
df_clip_info.shape

(21865, 4)

In [52]:
df_audio_info[df_audio_info['start_time'] < .isna()]

,id,clip,start_time,end_time


In [44]:
# df_clip_info = pd.DataFrame(columns=['id','clip','start_time','end_time'])
df_clip_info = pd.DataFrame()

part_ids = uniq_ids[:5]
#print(part_ids)
tmp_dict = {i : audio_clip_dict[i] for i in part_ids}
#print(tmp_dict)
clips_info_list = []
for i, clips in tmp_dict.items():
    mask_id = df_audio_info['id'] == i
    for clip in clips:
        mask_clip = df_audio_info['clip'] == clip
        clip_dict = {}
        clip_dict['id'] = i
        clip_dict['clip'] = clip
        clip_dict['start_time'] = \
            df_audio_info[(mask_id) & (mask_clip)]['start_time'].values[0]
        clip_dict['end_time'] = \
            df_audio_info[(mask_id) & (mask_clip)]['end_time'].values[0]
        clips_info_list.append(clip_dict)
df_clip_info = pd.DataFrame(clips_info_list)
display(df_clip_info)

,id,clip,start_time,end_time
0,--qXJuDtHPw,5,23.199,30.325
1,-3g5yACwYnA,2,4.840,14.052
2,-3g5yACwYnA,3,13.211,27.521
3,-3g5yACwYnA,4,26.541,41.300
4,-3g5yACwYnA,9,74.083,82.776
5,-3g5yACwYnA,10,82.753,100.555
6,-3g5yACwYnA,13,119.919,125.299
7,-3nNcZdcdvU,0,0.000,9.213
8,-3nNcZdcdvU,2,12.014,20.098
9,-571d8cVauQ,0,0.000,4.992


In [41]:
df_audio_info.dtypes

id             object
clip            int64
start_time    float64
end_time      float64
dtype: object

In [ ]:
def get_audio_file_names(data_dir):
    file_names = os.listdir(data_dir)
    file_names = sorted(file_names)
    return file_names

In [ ]:
def convert_to_mono(file_names, data_dir, out_dir):
    cnt = 0
    err_files = []
    for i in range(len(file_names)):
        print(f"file: {file_names[i]}")
        if subprocess.run([ffmpeg, "-i", data_dir+file_names[i], "-ac", "1",
                           out_dir+file_names[i], "-hide_banner"]).returncode != 0:
            print(f"file: {file_names[i]} error on conversion")
            err_files.append(file_names[i])
    print("\nConversion terminée")
    return err_files

In [ ]:
def get_names_and_convert_to_mono(data_dir, out_dir):
    start_time = ptime()
    raw_file_names = np.array(get_audio_file_names(data_dir))
    num_files = len(raw_file_names)
    # print(raw_file_names[:2])
    #err_files = convert_to_mono(raw_file_names[:2])
    err_files = convert_to_mono(raw_file_names, data_dir, out_dir)
    end_time = ptime()
    tot_time = round(end_time - start_time, 3)
    print(f"Temps de conversion pour {num_files} fichiers : {tot_time} sec.")
    print(f"Nombre d'erreurs de conversion: {len(err_files)}\n")
    print(err_files)